In [2]:
%load_ext autoreload
# %reload_ext autoreload

In [35]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from pycaret.regression import *
from pathlib import Path

In [38]:
%autoreload
import sys
sys.path.append(sys.path[0][0:-4] + 'mikael')
from pipeline import Pipin
pipin = Pipin()

In [5]:

dfa = pipin.get_data("A")
df_bc = pipin.get_combined_datasets(data_sets={"B", "C"})

In [7]:
to_ignore = []
keepers = [
"time",
"clear_sky_energy_1h:J",
"visibility:m",
"air_density_2m:kgm3",
"direct_rad_1h:J",
"date_calc",
"ceiling_height_agl:m",
"msl_pressure:hPa",
"pressure_100m:hPa",
"sfc_pressure:hPa",
"pressure_50m:hPa",
"sun_elevation:d",
"effective_cloud_cover:p",
"sun_azimuth:d",
"relative_humidity_1000hPa:p",
"cloud_base_agl:m",
"diffuse_rad:W",
"dew_point_2m:K",
"absolute_humidity_2m:gm3",
"precip_5min:mm",
"direct_rad:W",
"diffuse_rad_1h:J",
"prob_rime:p",
"t_1000hPa:K",
"clear_sky_rad:W",
"total_cloud_cover:p",
"elevation:m",
"dew_or_rime:idx",
"is_day:idx",
"is_in_shadow:idx",
"day_of_year",
"hour",
"month",
"year",
"pv_measurement",
"location:idx",
]


# add rest of name from df columns
for col in dfa.columns:
    if col not in keepers:
        to_ignore.append(col)

In [14]:
categorical_features =pipin.get_categorical_features(dfa)
categorical_features.remove("location:idx")
categorical_features

['dew_or_rime:idx',
 'is_day:idx',
 'is_in_shadow:idx',
 'precip_type_5min:idx',
 'snow_drift:idx']

In [15]:
ignore_features = pipin.get_irrelevant_features()
ignore_features

['date_calc', 'time', 'consecutive_count', 'location:idx']

In [19]:
a = setup(dfa, target="pv_measurement", normalize=True, categorical_features=categorical_features, experiment_name="pv_power_mw", ignore_features=ignore_features, index=False)
b = setup(df_bc, target="pv_measurement", normalize=True, categorical_features=categorical_features, experiment_name="pv_power_mw", ignore_features=ignore_features, index=False)

,Description,Value
0,Session id,8076
1,Target,pv_measurement
2,Target type,Regression
3,Original data shape,"(34085, 52)"
4,Transformed data shape,"(34085, 55)"
5,Transformed train set shape,"(23859, 55)"
6,Transformed test set shape,"(10226, 55)"
7,Ignore features,4
8,Ordinal features,2
9,Numeric features,43


,Description,Value
0,Session id,5896
1,Target,pv_measurement
2,Target type,Regression
3,Original data shape,"(32843, 52)"
4,Transformed data shape,"(32843, 55)"
5,Transformed train set shape,"(22990, 55)"
6,Transformed test set shape,"(9853, 55)"
7,Ignore features,4
8,Ordinal features,3
9,Numeric features,43


,Description,Value
0,Session id,5655
1,Target,pv_measurement
2,Target type,Regression
3,Original data shape,"(26095, 52)"
4,Transformed data shape,"(26095, 55)"
5,Transformed train set shape,"(18266, 55)"
6,Transformed test set shape,"(7829, 55)"
7,Ignore features,4
8,Ordinal features,2
9,Numeric features,43


In [29]:
lightgbm_a = a.create_model('lightgbm')
lightgbm_bc = b.create_model('lightgbm')


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,174.7822,143597.1323,378.9421,0.8917,1.1798,1.8149
1,183.4295,170869.6187,413.3638,0.8731,1.1425,1.0275
2,177.3661,164436.5422,405.5078,0.8719,1.3269,1.5965
3,181.0747,159684.4990,399.6054,0.8822,1.1646,1.1437
4,206.1556,195696.2067,442.3756,0.8610,1.1941,1.4679
5,181.4178,162154.0335,402.6835,0.8818,1.2056,1.7748
6,180.3000,161064.2198,401.3281,0.8838,1.2937,1.7043
7,185.2975,156867.6426,396.0652,0.8893,1.2248,1.0726
8,188.4650,167528.8965,409.3029,0.8752,1.1898,1.0529


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,35.0286,5091.3186,71.3535,0.8603,1.4856,1.2051
1,38.0333,6258.2831,79.1093,0.8460,1.4315,1.4319
2,35.4228,5330.8737,73.0128,0.8508,1.4980,1.7466
3,37.7408,6017.2261,77.5708,0.8439,1.5473,1.5666
4,34.4851,5205.2960,72.1477,0.8600,1.4741,1.6845
5,35.5128,5595.1931,74.8010,0.8440,1.4445,25.9078
6,33.8298,4627.7221,68.0274,0.8764,1.4784,1.4039
7,35.6526,5154.6399,71.7958,0.8709,1.5420,1.4543
8,35.4432,4989.1373,70.6338,0.8709,1.4970,2.0741


In [31]:
lightgbm_a_tuned = tune_model(lightgbm_a, optimize="MAE")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,20.6755,2396.0128,48.9491,0.9092,0.8154,0.7412
1,20.5522,2342.1173,48.3954,0.9187,0.8899,0.6338
2,21.4519,2533.0861,50.3298,0.9104,0.8413,0.6107
3,21.6717,2735.8216,52.3051,0.9078,0.8036,0.6881
4,21.6816,2879.9353,53.6650,0.8903,0.8217,0.5275
5,20.6600,2463.1186,49.6298,0.9079,0.8679,0.6430
6,22.9776,3180.3047,56.3942,0.8938,0.8529,0.5950
7,19.1591,2325.2804,48.2212,0.9086,0.8462,0.5467
8,21.3834,2786.2850,52.7853,0.8873,0.8285,0.7988


Fitting 10 folds for each of 10 candidates, totalling 100 fits
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightG

In [32]:
lightgbm_bc_tuned = tune_model(lightgbm_bc, optimize="MAE")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,20.6755,2396.0128,48.9491,0.9092,0.8154,0.7412
1,20.5522,2342.1173,48.3954,0.9187,0.8899,0.6338
2,21.4519,2533.0861,50.3298,0.9104,0.8413,0.6107
3,21.6717,2735.8216,52.3051,0.9078,0.8036,0.6881
4,21.6816,2879.9353,53.6650,0.8903,0.8217,0.5275
5,20.6600,2463.1186,49.6298,0.9079,0.8679,0.6430
6,22.9776,3180.3047,56.3942,0.8938,0.8529,0.5950
7,19.1591,2325.2804,48.2212,0.9086,0.8462,0.5467
8,21.3834,2786.2850,52.7853,0.8873,0.8285,0.7988


Fitting 10 folds for each of 10 candidates, totalling 100 fits
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightG

In [34]:
# tuned = s.tune_model(lightgbm)
final_a= a.finalize_model(lightgbm_a)
final_b= b.finalize_model(lightgbm_bc)
# final_c= c.finalize_model(lightgbm_c)


In [39]:
test_data_a = pipin.get_test_data("A")
test_data_b = pipin.get_combined_test_data(data_sets={"B", "C"})

In [23]:

def get_unique_filename(base_name, folder = "submissions"):
    end = ""
    if folder == "models":
        end = ".pkl"
    
    file_path = Path(folder) / f"{base_name}.csv{end}"
    count = 1
    while file_path.exists():
        file_path = Path(folder) / f"{base_name}_{count}.csv{end}"
        count += 1
    return str(file_path)

In [40]:
res_stack_a = predict_model(final_a, test_data_a)
res_stack_b = predict_model(final_b, test_data_b)
concatinated = pd.concat([res_stack_a, res_stack_b]).reset_index(drop=True)


concatinated

,time,sun_azimuth:d,pressure_100m:hPa,fresh_snow_1h:cm,visibility:m,precip_5min:mm,direct_rad:W,diffuse_rad:W,wind_speed_10m:ms,total_cloud_cover:p,...,dew_or_rime:idx,is_day:idx,is_in_shadow:idx,precip_type_5min:idx,snow_drift:idx,day_of_year,hour,month,location:idx,prediction_label
0,2023-05-01 00:00:00,16.026501,1000.549988,0.0,29907.500000,0.0,0.000000,0.000000,3.950,74.949997,...,0.0,0.0,1.0,0.0,0.0,121,0,5,A,3.108333
1,2023-05-01 01:00:00,30.497250,1000.049988,0.0,29519.074219,0.0,0.000000,0.000000,3.825,77.474998,...,0.0,0.0,1.0,0.0,0.0,121,1,5,A,3.108333
2,2023-05-01 02:00:00,44.517250,999.500000,0.0,31009.125000,0.0,0.000000,0.000000,3.650,88.099998,...,0.0,0.0,1.0,0.0,0.0,121,2,5,A,3.108333
3,2023-05-01 03:00:00,58.083000,998.974976,0.0,34552.500000,0.0,2.100000,9.375000,3.500,68.599998,...,0.0,1.0,0.0,0.0,0.0,121,3,5,A,50.222725
4,2023-05-01 04:00:00,71.341003,998.549988,0.0,35483.875000,0.0,25.450001,47.400002,3.325,66.300003,...,0.0,1.0,0.0,0.0,0.0,121,4,5,A,313.036242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,2023-07-03 19:00:00,306.688995,977.575012,0.0,41007.898438,0.0,13.450001,32.200001,2.175,88.175003,...,0.0,1.0,0.0,0.0,0.0,184,19,7,C,62.702372
2156,2023-07-03 20:00:00,319.703003,977.974976,0.0,41315.949219,0.0,2.400000,13.875000,2.200,78.949997,...,0.0,1.0,0.0,0.0,0.0,184,20,7,C,39.681694
2157,2023-07-03 21:00:00,333.037231,978.250000,0.0,41665.898438,0.0,0.000000,1.250000,2.250,80.449997,...,0.0,0.0,1.0,0.0,0.0,184,21,7,C,30.070626
2158,2023-07-03 22:00:00,346.682739,978.650024,0.0,39007.601562,0.0,0.000000,0.000000,1.875,100.000000,...,0.0,0.0,1.0,0.0,0.0,184,22,7,C,38.541425


In [41]:
res_stack = concatinated[["prediction_label"]].rename(columns={"prediction_label": "prediction"}).rename_axis(index="id")

res_stack.to_csv(get_unique_filename('a_model_and_bc_model', 'submissions'), index=True)
# save_model(stacker, model_name=get_unique_filename('stack_gbm_et_rf', "models"))

In [43]:
print("done", res_stack.shape)
pipin.compare_mae(res_stack)


done (2160, 1)


61.693566189292

In [ ]:
# 